In [11]:
import nltk
import numpy as np
import json
import random
import urllib
from nltk.stem.lancaster import LancasterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
file=open("C:\\Users\\user\\Desktop\\Wing_ML-Chat\\Wing_ML_Chat.json.txt").read()
data = json.loads(file)

In [13]:
docs_x = []
docs_y = []

In [14]:
stemmer = LancasterStemmer()
for intent in data['intents']:
	for pattern in intent['patterns']:
		wrds = nltk.word_tokenize(pattern)
		wrds = [stemmer.stem(w.lower()) for w in wrds if w != "?"]
		docs_x.append(' '.join(wrds))
		docs_y.append(intent["tag"])

In [15]:
cv = CountVectorizer()
X = cv.fit_transform(docs_x).toarray()
print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [16]:
le = LabelEncoder()
y = le.fit_transform(docs_y)

In [17]:
from sklearn.svm import SVC
classifier = SVC(kernel='linear')
classifier.fit(X, y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [18]:

def provide_response(sent):
  #Pre-Process Input
  sent = nltk.word_tokenize(sent)
  sent = [stemmer.stem(w.lower()) for w in sent if w != "?"]
  sent = [' '.join(sent)]
  X_pred = cv.transform(sent).toarray()

  #Predict
  y_pred = classifier.predict(X_pred)
  tag = le.inverse_transform(y_pred)
  tag = tag[0]

  #Get the options to respond
  for tg in data['intents']:
    if tg['tag'] == tag:
      responses = tg['responses']

  #Return one of them
  return (tag, random.choice(responses))

In [19]:
print(provide_response("What can you do?"))

('introduction', 'I am MLChat. I can provide you answers to your wing related Questions')


In [20]:
print ("BOT : Hello! I am ML Chat.")

while True:
  inp = input("\nYOU : ")
  resp = provide_response(inp)
  
  print("\nBOT : " + resp[1])
  
  if resp[0] == "goodbye" : break

BOT : Hello! I am ML Chat.

YOU : wingbox

BOT : Top skin, Bottom skin, FS, RS, LE, Wing-tip

YOU : wing

BOT : Top skin, Bottom skin, FS, RS, LE, Wing-tip

YOU : damage

BOT : corrosion, dent, gouge,  scratch

YOU : define damage

BOT : corrosion, dent, gouge,  scratch

YOU : materials

BOT : Top skin- 7150-T651, bottom skin - 2024-T351,  spar web- 2024-T42

YOU : top panel

BOT : Top skin- 2 panels, str 11 buttstrap, made of 7150-7561, critcal for compression

YOU : buttstrap

BOT : Top skin, Bottom skin, FS, RS, LE, Wing-tip

YOU : see you

BOT : Bye! Come back again soon.
